In [1]:
import json

In [2]:
with open("../data/first_batch_categorization.json") as f, open("../data/first_batch_span_fixing.json") as g:
    cat = sorted(json.load(f), key=lambda x: x["ID"])
    span = sorted(json.load(g), key=lambda x: x["ID"])

## Solve the problem through a mapping without understanding it

In [3]:
import copy
span_new = copy.deepcopy(span)

# For each HIT
for i in range(len(span_new)):
    # created a combined list of all cat sentences
    cat_combined = cat[i]['Deletions'] + cat[i]['Paraphrases'] + cat[i]['Splittings']
    cat_combined_systems = [x[1] for x in cat_combined]

    # For each category in {Deletions, Paraphrases, Splittings}
    for category in ['Deletions', 'Paraphrases', 'Splittings']:        
        # For each span sentence in category
        for j in range(len(span_new[i][category])):
            # Look for the setence in cat_combined where the sources of the sentences match
            corrected = cat_combined[cat_combined_systems.index(span_new[i][category][j][1])]
            
            # Copy the contents of the sentence and the edit from cat_combined to span
            span_new[i][category][j] = corrected

In [4]:
with open('../data/first_batch_span_fixing_fixed.json', 'w') as f:
    json.dump(span_new, f)

In [5]:
# Use this if we want to see whether our correction code is correct
span = copy.deepcopy(span_new)

## Understand the errors being created

In [6]:
for i in range(len(cat)):
    print(f'Checking HIT {i}...')
    cat_sent = cat[i]['Deletions'] + cat[i]['Paraphrases'] + cat[i]['Splittings']
    span_sent = span[i]['Deletions'] + span[i]['Paraphrases'] + span[i]['Splittings']
    
    # Amount of sentences are the same
    if (len(cat_sent) != len(span_sent)):
        print("arrays different lengths")
        
    cat_sorted = sorted(cat_sent, key=lambda x: x[0])
    span_sorted = sorted(span_sent, key=lambda x: x[0])
    
    # Contents of sentences are the same
    cat_sent_only = [x[0] for x in cat_sorted]
    span_sent_only = [x[0] for x in span_sorted]
    for j in range(len(cat_sent_only)):
        if (cat_sent_only[j] != span_sent_only[j]):
            print(f"{j} is different")
            
    # Edits are the same, even if they are out of order
    for j in range(len(cat_sent)):
        cat_edits = sorted(cat_sorted[j][2:], key=lambda x: x[1])
        span_edits = sorted(span_sorted[j][2:], key=lambda x: x[1])
        if (len(cat_edits) != len(span_edits)):
            print(f'Different amounts of edits on sent {j}: \n\t {cat_edits} \n\t {span_edits}') #\n\t on  {cat_sent[j]} \n\t and {span_sent[j]}')        
        else:
            for k in range(len(cat_edits)):
                for l in range(len(cat_edits[0])):
                    if (cat_edits[k][l] != span_edits[k][l]):
                        print(f'Error contents of edit have changed on sent {j}')
print('Done!')

Checking HIT 0...
Checking HIT 1...
Checking HIT 2...
Checking HIT 3...
Checking HIT 4...
Checking HIT 5...
Checking HIT 6...
Checking HIT 7...
Checking HIT 8...
Checking HIT 9...
Checking HIT 10...
Checking HIT 11...
Checking HIT 12...
Checking HIT 13...
Checking HIT 14...
Checking HIT 15...
Checking HIT 16...
Checking HIT 17...
Checking HIT 18...
Checking HIT 19...
Done!


### For testing

In [7]:
i = 0
cat_sent = cat[i]['Deletions'] + cat[i]['Paraphrases'] + cat[i]['Splittings']
span_sent = span[i]['Deletions'] + span[i]['Paraphrases'] + span[i]['Splittings']

In [8]:
# Amount of sentences are the same
if (len(cat_sent) != len(span_sent)):
    print("arrays different lengths")

# Contents of sentences are the same
cat_sorted = sorted([x[0] for x in cat_sent])
span_sorted = sorted([x[0] for x in span_sent])
for j in range(len(cat_sorted)):
    if (cat_sorted[j] != span_sorted[j]):
        print(f"{j} is different")

In [9]:
# Edits are the same, even if they are out of order
for j in range(len(cat_sent)):
    cat_edits = sorted(cat_sent[j][2:], key=lambda x: x[1])
    span_edits = sorted(span_sent[j][2:], key=lambda x: x[1])
    if (len(cat_edits) != len(span_edits)):
        print(f'Different amounts of edits on sent {j}: \n\t {cat_edits} \n\t {span_edits}') #\n\t on  {cat_sent[j]} \n\t and {span_sent[j]}')        
    else:
        for k in range(len(cat_edits)):
            for l in range(len(cat_edits[0])):
                if (cat_edits[k][l] != span_edits[k][l]):
                    print(f'Error contents of edit have changed on sent {j}')